# 🔗 유불리 판단 + 유사 조항 검색 + Ollama 기반 LLM 설명 생성

In [1]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama


c:\Users\user\Documents\JSY\KW\3-1\TextMining\TextMining\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 유불리 판단 모델
classifier_model = AutoModelForSequenceClassification.from_pretrained("classification_model")
classifier_tokenizer = AutoTokenizer.from_pretrained("classification_model")

# 의미 임베딩 모델
semantic_model = SentenceTransformer("legal-kr-sbert-contrastive")

# 임베딩 데이터셋 로드
dataset = torch.load("embedding_dataset.pt", map_location=torch.device('cpu'))
texts = dataset["texts"]
embeddings = dataset["embeddings"]


In [3]:
def predict_unfairness(clauses):
    inputs = classifier_tokenizer(clauses, padding=True, truncation=True, return_tensors="pt")
    outputs = classifier_model(**inputs)
    preds = torch.argmax(outputs.logits, dim=1)
    return preds.tolist()  # 0: 유리, 1: 불리

In [4]:
import pandas as pd

# labeled.csv 읽기
df = pd.read_csv("labeled.csv")  # 또는 정확한 경로

In [5]:
def get_similar_clauses(query, top_k=5):
    query_emb = semantic_model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_emb, embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    results = []
    for idx in top_results.indices:
        clause = texts[idx]
        # labeled.csv에서 basis 찾기
        matched = df[df["text"] == clause]
        basis = matched["basis"].values[0] if not matched.empty else ""
        results.append({"clause": clause, "basis": basis})
    
    return results


In [6]:
# Ollama LLM 설정
llm = ChatOllama(model="anpigon/EEVE-Korean-10.8B:latest")

# 프롬프트 템플릿
prompt_template = PromptTemplate(
    input_variables=["clause", "similar"],
    template="""다음은 서비스 약관의 조항입니다:

조항:
{clause}

유사한 조항들:
{similar}

이 조항이 왜 불리한지 설명해 주세요.
"""
)

llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# 설명 생성 함수
def generate_explanation(clause, similar_clauses):
    similar_text = "\n\n".join(
        [f"- 조항:\n{item['clause']}\n설명:\n{item.get('basis', '')}" for item in similar_clauses]
    )

    return llm_chain.run({
        "clause": clause,
        "similar": similar_text
    })

C:\Users\user\AppData\Local\Temp\ipykernel_14284\2084197299.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="anpigon/EEVE-Korean-10.8B:latest")
C:\Users\user\AppData\Local\Temp\ipykernel_14284\2084197299.py:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_template)


In [7]:
print("📥 여러 약관 조항을 한 줄씩 입력하세요. (빈 줄 입력 시 종료)\n")
input_clauses = []
while True:
    line = input("조항: ")
    if line.strip() == "":
        break
    input_clauses.append(line)

labels = predict_unfairness(input_clauses)

for clause, label in zip(input_clauses, labels):
    if label == 1:
        similar = get_similar_clauses(clause, top_k=5)
        explanation = generate_explanation(clause, similar)

        print(f"\n🔹 조항: {clause}")
        print("📎 유사 조항:")
        for i, s in enumerate(similar, 1):
            print(f"{i}. {s}")
        print(f"\n🧠 설명:\n{explanation}")

    else:
        print(f"\n🔹 조항: {clause}")
        print("✅ 유리한 조항으로 판단됨")

📥 여러 약관 조항을 한 줄씩 입력하세요. (빈 줄 입력 시 종료)



C:\Users\user\AppData\Local\Temp\ipykernel_14284\2084197299.py:27: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm_chain.run({



🔹 조항: 제10조(포인트 정정, 취소, 소멸)\n제5항 본 조 제3항에도 불구하고 본 약관 제7조 제1항 및 제2항에 따라 카드 탈퇴 및 자격상실된회원은 포인트가 소멸되고, 이에 대하여 회원은 어떠한 권리 주장할 수 없습니다. \n000의 경우, 기존 포인트 약관상  카드 해지 와  탈회 를 별도로 구분하지 아니하고,  탈퇴 라는 포괄적인 용어를 사용하였음.
📎 유사 조항:
1. {'clause': '(000 회원약관)\n제10조(포인트 정정, 취소, 소멸)\n제5항 본 조 제3항에도 불구하고 본 약관 제7조 제1항 및 제2항에 따라 카드 탈퇴 및 자격상실된 회원은 포인트가 소멸되고, 이에 대하여 회원은 어떠한 권리 주장할 수 없습니다.', 'basis': '특정 카드를 해지한 고객은 유효한 다른 카드를 이용하여 여전히 잔여포인트를 정상적으로 사용할 수 있는바, 해지 카드의 잔여포인트의 경우 해지 전 포인트와 그 취급을 달리할 특별한 이유가 존재하지 아니한다.'}
2. {'clause': '제9조(현금서비스)\n제5항 현금서비스금액과 동 수수료, 기타 이용수수료의 결제시기 및 방법은 제14조의 카드이용대금 결제방식과 같습니다.\n제14조(대금결제)\n제1항  회원은 카드이용대금과 이에 수반되는 모든 수수료를 카드사가 정하는 대금결제일에 자동대체 결제방법 또는 카드사 또는 카드사와 제휴한 기관에 직접납부하는 방법에 의하여 결제하여야 하며 대금결제일은 결제가능일 중에서 회원이 원하는 날로 지정할 수 있습니다.', 'basis': '현금서비스는 카드회원에게 단기간 소액의 금전을 간편하게 융자해주고자 하는 특수목적이 있기는 하나, 그 실질은 대출기간에 따라 수수료를 차등 있게 부과하는 금전사용대차계약에 해당되므로 상환액이나 상환방법 등에 있어서 일반금융대출과 달리 취급될 이유는 없다.  현금서비스의 경우는 일반금융대출에 비해 고율의 수수료가 부과되고 있으므로 고객들의 중도상환 요구가 많을 뿐만 아니라 상환에 따른 실익 또한 크다. 고객이 현금서비스를 받은 후 결제